## Sources of biological data

**What are databases?**

**Databases** are organized collections of structured information retrieved from various sources such as experiments, sensors, telemetry data and publications, to name a few. Information within the database is physically stored in computer systems and a set of programs manage how data flows in and out of the database. A **server** actively listens for *requests* sent by other computers to access the data. Upon validation, the server retrieves the data from the database and sends the data (*response*) to the requesting computer (or **client**).

**Biological databases**

In biology, high-throughput experiments had led to the generation of data in the scale of petabytes. An organizational scheme is needed to easily retrieve the data used for the generation of actionable insights, which is the ultimate goal of any analysis. **Biological databases** contain data generated from experiments in the fields as genomics, proteomics, metabolomics, epigenetics, and many more. Each data point reflects a particular attribute of a biological entity such as the function of a gene, structure of a protein, expression of a transcript, localization of mutation, etc.

**Why should I care?**

Prior to any analysis, data pertinent to the research question must first be collected in a useable form. Retrieving data may be as simple a going to NCBI, searching for a particular accession or query, and clicking a button to download a file in a specific format (e.g., FASTA, GFF). However, this approach is not scalable when attempting to process largescale data. As such, bioinformaticians have developed infrastructure for streamlining the process of data collection, allowing researchers from all backgrounds to retrieve the information they require in a straighforward manner.

The aim of this write-up is to introduce you to a few command-line-based tools for downloading biological data. 

## How to get genome data

Data is distributed via various repositories. The most commonly used ones are:

<center>

```{mermaid}
%%| fig-width: 100%
flowchart LR
  1A["`**NCBI**
  Genbank/RefSeq`"]
  1B["`Use numbers
  **bio, efetch**`"]
  
  2A["`**Ensembl**
  **UCSC**
  **FlyBase, WormBase**
  Releases`"]
  2B["`Use URL
  **curl, wget, aria2**
  **genomepy, refgenie**`"]
  
  3A["`**NCBI Assemblies**
  RefSeq genomes`"]
  3B["`Use accession
  **datasets**`"]
  
  4A["`**Independent
  Data Tools**`"]
  4B["`**genomepy**
  **refgenie**`"]
  
  5["`**File Formats**
  FASTA, GFF, GTF, BED`"]
  
  1A --> 1B --> 5
  2A --> 2B --> 5
  3A --> 3B --> 5
  4A --> 4B --> 5
```

</center>

In addition, there are software packages such as `refgenie` and `genomepy` that can be used to download and manage reference genomes.

### Search for metadata

The `bio` package is a CLI-based tool used for bioinformatics exploration. It contains commands for downloading, manipulating, and transforming sequence data. If you have an NCBI-based accession number, you can use the `bio search` command to get information on it.

```sh
# Use a GenBank accession
bio search AF086833      # <1>

# Use an SRA accession
bio search SRR1553425    # <2>

# Use a RefSeq assembly ID
bio search GCA_000005845 # <3>

# Use a query string
bio search ecoli         # <3>
```

1. Searches GenBank
2. Searches SRA
3. Searches NCBI Genome

Running the `bio search` command will return a JSON-formatted string which contains the metadata for a particular record. Use the `--csv` flag to output the metadata in a comma-delimited format. Similarly, use the `--csv` if you want tab-delimited data. 

Running `bio search SRR1553425` would produce the following:

In [ ]:
#| echo: false
#| eval: true

! micromamba run -n bioinfo bio search SRR1553425

We can then use the `jq` processing tool to extract fields of interest. Extract the `fastq_url` list by running:

In [ ]:
#| echo: true
#| eval: true
! bio search SRR1553425 | jq ".[].fastq_url[]"

### Accessing Genbank

GenBank is the NIH genetic sequence database, an annotated collection of publicly available DNA sequences. If your data has a GenBank accession number such as `AF086833`, use the `bio fetch` command. By default, data is printed to stdout. Override this behavior by specifying the output filename with the `-o` flag or redirect the output to a file with the `>` operator.

```bash
# Accession id pointing to the record.
ACC=AF086833                                # <1>

# Specify output with a flag.
bio fetch ${ACC} --format fasta -o ${ACC}.fa # <2>

# Redirect the output to a file.
bio fetch ${ACC} --format gff > ${ACC}.gff  # <3>
```
1. Store accesssion ID as a variable.
2. Download the sequence (FASTA) file.
3. Download the annotation (GFF) file.

Let us verify the download by viewing the first ten lines of the annotation file:

In [ ]:
#| echo: false
#| eval: true

! bio fetch AF086833 --format gff | head -n 10

### Download via URL

If you know the URL of a resource, you may use `wget` or `curl` to download the file. First, save the URL to a variable for referencing:
```bash
URL=http://ftp.ensembl.org/pub/release-104/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz
```

Use `curl`:
```bash
curl ${URL} -o chr22.fa.gz
```
  
Or use `wget`:
```bash
wget -nc ${URL} -o chr22.fa.gz
```

The `-nc` flag would skip the download altogether if the file already exists. Use this flag to ensure that large files are not overwritten.

For large files, [aria2](https://aria2.github.io/) can be used for faster, multi-segmented downloads. The tool also supports checkpoints which allow you to restart interrupted downloads. Download aria2 from their website or use `conda` to install in an environment.
```bash
aria2c ${URL} \ 
  -x 5 \           # <1>
  -o chr22.fa.gz   # <2>
```
1. Connect to the server with x connections.
2. Save the output to a file.


### How to use NCBI datasets

[NCBI Datasets](https://www.ncbi.nlm.nih.gov/datasets/docs/v2/download-and-install/) is a new resource designed by NCBI to gather data from across NCBI databases. The main entry point of the tool is the `datasets` command. Subcommands such as `download` or `summary` is then specified, followed by more subcommands to specify your query.

It seems to be the direction that NCBI wants to shepherd users towards. However, the nested structure of running the tool makes its use complicated and convoluted compared to other resources. NCBI is kind enough to give us a diagram for navigating the subcommands:

<center>

![](https://www.ncbi.nlm.nih.gov/datasets/docs/v2/datasets_schema_taxonomy.svg)

</center>

Swiftwater hydra (*Hydra vulgaris*) has a taxonomy id of `6087` and RefSeq id of `GCF_038396675.1`. We can download its genome by running the following:
```sh
datasets download genome accession GCF_038396675.1
```

By default, genome is downloaded as a zipped file named **ncbi_dataset**. The structure of the directory is seen below.
```
ncbi_dataset
└── data
    ├── assembly_data_report.jsonl
    ├── dataset_catalog.json
    └── GCF_038396675.1
        └── GCF_038396675.1_HydraT2T_AEP_genomic.fn
```

### How to access Ensembl

Ensembl operates on numbered releases. For example, [release 104](http://ftp.ensembl.org/pub/release-104/) was published on March 30, 2021. Data can be retrieved by navigating the file tree in the provided FTP server. Alternatively, you can invoke `curl`, `wget`, or `aria2c` directly on each file.

```sh
# Get the FASTA file from chromosome 22 of the human genome
URL=http://ftp.ensembl.org/pub/release-104/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz

# Use curl go download the data
curl ${URL} | gunzip -c > refs/chr22.fa # <1>
```
1. Download from URL and decompress the FASTA file.

## How to use refgenie

Refgenie is a command-line tool that can be used to download and manage reference genomes, and to build and manage custom genome assets. It also provides a Python interface for programmatic access to genome assets.

<center>

```{mermaid}
flowchart LR
  list["`**refgenie**
  list`"]
  pull["`**refgenie**
  pull`"]
  seek["`**refgenie**
  seek`"]
  list --> pull --> seek
```

</center>

### Installation

Refgenie can be installed as a Python package using `pip`:

```bash
# Install using pip.
pip install refgenie

# Install using pipx.
pipx install refgenie
```

or conda:

```sh
conda install -c conda-forge refgenie

# Use mamba/micromamba instead of conda
micromamba install refgenie
```

### Create a config file

`refgenie` requires a configuration file that lists the resources in the form of a yaml file. For that you need to select a directory that will store the downloaded resources. The path to the config file is saved as the `REFGENIE` shell environment variable which will be used for initialization:

```bash
# Path pointing to refgenie config file.
export REFGENIE=~/refs/config.yaml

# Load the REFGENIE variable when launching a shell instance
echo "export REFGENIE=~/refs/config.yml" >> ~/.bashrc
source ~/.bashrc

# Run initialization
refgenie init
```

The `refgenie` tool is now ready to be used to download and manage reference genomes.

### Using refgenie

A list of pre-built assets from a remote server can be displayed with `listr`:

In [ ]:
#| echo: true
#| eval: true

! refgenie listr

Genome data is fetched using the `pull` command:
```bash
refgenie pull hg38/bwa_index
```

The `seek` command displays the path of the downloaded file:
```bash
refgenie seek hg38/bwa_index
```

List local genome assets:
```bash
refgenie list
```

Use command substitution to store the genome path to a variable:
```bash
# Retrieve the human reference genome
refgenie pull hg38/fasta

# Save path of reference genome to a variable
REF=$(refgenie seek hg38/fasta)
```

Then use the resulting path in downstream tools:
```sh
# Generate statistics for the human reference genome
seqkit stats ${REF}
```

Subscribe to the iGenomes server which hosts additional reference genomes and genome assets.

```sh
refgenie subscribe -s http://igenomes.databio.org/
```

## Using genomepy

### How to use genomepy

`genomepy` is another tool designed for searching and downloading genomic data. It can be used to:

1. search available data
2. show the available metadata
3. automatically download, preprocess
4. generate optional aligned indexes

Currently, `genomepy` supports Ensembl, UCSC, NCBI, and GENCODE.

<center>

```{mermaid}
flowchart LR
  genomepy["`**genomepy**`"]
  commands{"`**search**
  **install**
  **annotation**`"}
  storage["`files stored in
  **$home/local/share/genomes**`"]
  
  genomepy --> commands --> storage
```

</center>

See also: [genomepy documentation](https://github.com/vanheeringen-lab/genomepy)

### Installation

Install using micromamba:
```bash
micromamba install genomepy
```

Install using pip or pipx:
```bash
pipx install genomepy
```

### Using genomepy

Use the `search` command to query genomes by name or accession:
```bash
genomepy search ecoli > ecoli_query_results.txt
```

A genome index will be downloaded upon invoking the `search` command for the first time. Hence, the initial search may take a while depending on your connection speed. As seen from the log below, assembly summaries are fetched from multiple databases (GENCODE, UCSC, Ensembl, NCBI).

```md
05:28:31 | INFO | Downloading assembly summaries from GENCODE
05:29:54 | INFO | Downloading assembly summaries from UCSC
05:30:05 | INFO | Downloading assembly summaries from Ensembl
05:30:43 | INFO | Downloading assembly summaries from NCBI, this will take a while...
genbank_historical: 73.0k genomes [00:06, 11.1k genomes/s]
refseq_historical: 85.6k genomes [00:05, 16.6k genomes/s]
genbank: 2.39M genomes [02:11, 18.1k genomes/s]
refseq: 378k genomes [00:28, 13.0k genomes/s] 
```

The results look like so:
```md
name                 provider accession         tax_id annotation species                                  other_info                              
                                                        n r e k   <- UCSC options (see help)                                                       
EcoliT22_2.0         NCBI     GCF_000247665.3      562     ✓      Escherichia coli O157:H43 str. T22       BAYGEN                                  
Ecoli.O104:H4.LB226692_2.0 NCBI     GCA_000215685.3      562     ✓      Escherichia coli O104:H4 str. LB226692   Life Technologies                       
Ecoli.O104:H4.01-09591_1.0 NCBI     GCA_000221065.2      562     ✓      Escherichia coli O104:H4 str. 01-09591   Life Technologies                       
Ecoli_C227-11_1.0    NCBI     GCA_000220805.2      562     ✓      Escherichia coli O104:H4 str. C227-11    PacBio                                  
ecoli009             NCBI     GCA_900607665.1      562     ✓      Escherichia coli                         BIOZENTRUM, UNIVERSITY OF BASEL         
ecoli006             NCBI     GCA_900607465.1      562     ✓      Escherichia coli                         BIOZENTRUM, UNIVERSITY OF BASEL         
ecoli008             NCBI     GCA_900607535.1      562     ✓      Escherichia coli                         BIOZENTRUM, UNIVERSITY OF BASEL         
ecoli017             NCBI     GCA_900608025.1      562     ✓      Escherichia coli                         BIOZENTRUM, UNIVERSITY OF BASEL         
ecoli025             NCBI     GCA_900608175.1      562     ✓      Escherichia coli                         BIOZENTRUM, UNIVERSITY OF BASEL         
ecoli022             NCBI     GCA_900608105.1      562     ✓      Escherichia coli                         BIOZENTRUM, UNIVERSITY OF BASEL         
ecoli015             NCBI     GCA_900607975.1      562     ✓      Escherichia coli                         BIOZENTRUM, UNIVERSITY OF BASEL         
ecoli013             NCBI     GCA_900607805.1      562     ✓      Escherichia coli                         BIOZENTRUM, UNIVERSITY OF BASEL       
```

Entries under the `name` field can be used to download the genome:
```bash
genomepy install ecoli009
```

By default, the downloaded genomes will be found in the `~/.local/share/genomes` directory. For our example, the directory named `ecoli009` contains the genome data and other relevant files:

In [ ]:
#| echo: false
#| eval: true

! tree ~/.local/share/genomes

## How to get FASTQ data

Publibashed FASTQ files are stored in the bashort Read Archive (SRA). Access to SRA can be diagrammed like so:

<center>

```{mermaid}
flowchart LR
  fq["`**FASTQ FILES**`"]
  srr["`**SRR number**`"] --> srr2["`Find URL and metadata
  web, **bio**, **ffq**`"]
  sra["`**SRA**
  bashort Read Archive`"] --> sra2["`Use SRR number
  **fastq-dump**`"] --> fq
  ensembl["`**Ensembl**
  Sequence archive`"] --> ensembl2["`Find URL
  **curl**, **wget**, **aria2**`"]
  com["`**Commercial**
  Google, Amazon
  Users pay to download`"] --> com2["`Custom tools
  **gsutil**, **aws**`"] --> fq
```

</center>

The sratools suite from NCBI provides `fastq-dump` and `fasterq-dump` to download read data from SRA accessions. In later versions of sratools, `fasterq-dump` is the preferred tool for fetching read data as demonstrated below:

```bash
# Store accession number and number of reads
ACC=SRR1553425
N=10000

# Create reads directory
mkdir reads

# Fetch reads from accession
fasterq-dump --split-3 -X ${N} -O reads ${ACC}
```

However this method is clunky and fragile, often failing to fetch the required data due to errors that are cryptically communicated to the user. An alternative method is to retrieve the URLs that point to the data and download locally using `wget`, `curl` or `aria2`. Use `bio search` to retrieve metadata on the SRA accession and parse using `jq`.

In [ ]:
#| echo: true
#| eval: true

! bio search SRR1553425 | jq -r '.[].fastq_url[]'

### Using the SRA Explorer

The **SRA Explorer** is a web-based tool developed by [Phil Lewis](http://phil.ewels.co.uk/) aimed to make SRA data more accessible. It allows you to search for datasets and view metadata. The link can be accessed here:

- https://sra-explorer.info/

### Using the NCBI website

You can also visit NCBI's SRA repository [here](https://sra-explorer.info/) to download sequencing read data. 

### How to download multiple runs

All data from a project can be queried using `bio search`, parsed using `csvcut`, and concurrently downloaded using `parallel` or `aria2c`:

The project id encapsulates all the details in a sequencing experiment. Pass the project id as an argument to the `bio search` command to view the SRR accessions.
```bash
# Access the project metadata and save as a CSV file
bio search PRJNA257197 --csv > project.csv
```

The truncated output is as follows:

In [ ]:
#| echo: false
#| eval: true

! bio search PRJNA257197 --csv | head -n 5

Only the accession numbers are needed to download the reads. From the project file, we extract the first column corresponding to the accession, and use this as input to `fastq-dump`. The `parallel` tool enables us to simultaneously download multiple accession at once.
```bash
# Extract the first column and download concurrently using parallel
cat project.csv | \                                   # <1>
    csvcut -c 1 | \                                   # <2>
    head -n 3 | \                                     # <3>
    parallel "fastq-dump -F --split-files -O data {}" # <4>
```
1. Print to standard output.
2. Filter only the first column.
3. Filter first three rows.
4. Download reads for each of the three accessions.
